In [1]:
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer

In [2]:
def preprocessing_text(texts):
    texts = re.sub(r'<.*?>', '', texts)
    texts = re.sub(r'[^a-zA-Z]', ' ', texts)
    return ' '.join(x.lower() for x in texts.split())

In [3]:
df = pd.read_csv("IMDB Dataset.csv")

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
df['review_cleaned'] = df['review'].apply(lambda x : preprocessing_text(x))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df['review_cleaned'], df['sentiment'].map({'negative':0, 'positive':1}))

In [6]:
df['review_cleaned_500'] = df['review_cleaned'].apply(lambda x:x[:500])

df['review_cleaned_500'][2]

'i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a light hearted comedy the plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer while some may be disappointed when they realize this is not match point risk addiction i thought it was proof that woody allen is still fully in control of the style many of us have grown to love this was the most i d laugh'

In [7]:
tokenizer = Tokenizer(num_words=10000)

In [8]:
tokenizer.fit_on_texts(df['review_cleaned_500'])
seq = tokenizer.texts_to_sequences(df['review_cleaned_500'])
X = pad_sequences(seq, padding='post')

print(f'X_shape: {X.shape}, X_min: {np.min(X)}, X_max: {np.max(X)}')

X_shape: (50000, 116), X_min: 0, X_max: 9999


In [9]:
y = df['sentiment'].map({'negative' : 0, 'positive' : 1}).values

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13)

In [11]:
n_words = X_train.shape[1]

In [12]:
model = Sequential()
model.add(Dense(50, input_shape=(n_words,), activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile network
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
history = model.fit(X_train, y_train, epochs = 20, batch_size = 64, validation_data = (X_test, y_test))

Epoch 1/20
586/586 [==============================] - 2s 3ms/step - loss: 243.3105 - accuracy: 0.5000 - val_loss: 55.1101 - val_accuracy: 0.5045
Epoch 2/20
586/586 [==============================] - 2s 3ms/step - loss: 36.6518 - accuracy: 0.5179 - val_loss: 10.2884 - val_accuracy: 0.4985
Epoch 3/20
586/586 [==============================] - 2s 3ms/step - loss: 8.5216 - accuracy: 0.5131 - val_loss: 6.2898 - val_accuracy: 0.5074
Epoch 4/20
586/586 [==============================] - 2s 3ms/step - loss: 5.2541 - accuracy: 0.5077 - val_loss: 4.2644 - val_accuracy: 0.5026
Epoch 5/20
586/586 [==============================] - 1s 2ms/step - loss: 4.7215 - accuracy: 0.5021 - val_loss: 3.3641 - val_accuracy: 0.5037
Epoch 6/20
586/586 [==============================] - 1s 2ms/step - loss: 3.5354 - accuracy: 0.5059 - val_loss: 3.8438 - val_accuracy: 0.5005
Epoch 7/20
586/586 [==============================] - 1s 2ms/step - loss: 3.6348 - accuracy: 0.5117 - val_loss: 2.7203 - val_accuracy: 0.4964
E